<a href="https://colab.research.google.com/github/Sanskar6877/ChestXRay_Pneumonia_Detection/blob/main/Model3_ui_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# re-size all the images
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/chest_xray/train'
valid_path = '/content/drive/MyDrive/chest_xray/val'
test_path = '/content/drive/MyDrive/chest_xray/test'

In [ ]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# creating a model object
model = Model(inputs=resnet.input, outputs=prediction)


In [ ]:
print(prediction[0])

KerasTensor(type_spec=TensorSpec(shape=(2,), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   featurewise_center=False,  # set input mean to 0 over the dataset
                                  samplewise_center=False,  # set each sample mean to 0
                                 featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                 samplewise_std_normalization=False,  # divide each input by its std
                                 zca_whitening=False,  # apply ZCA whitening
                              rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
                             zoom_range = 0.2, # Randomly zoom image
                            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                           height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                         horizontal_flip = True,  # randomly flip images
                        vertical_flip=False)

val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen=ImageDataGenerator(rescale=1./255)

# providing the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')



val_set = val_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')



Found 5216 images belonging to 2 classes.
Found 220 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# fiting the model
r = model.fit(
  training_set,
  validation_data=val_set,
  epochs=10, #10,15,20
  steps_per_epoch=len(training_set),
  validation_steps=len(val_set)
)

Epoch 1/10
163/163 [==============================] - 1815s 11s/step - loss: 0.6844 - accuracy: 0.7621 - val_loss: 0.9186 - val_accuracy: 0.5636
Epoch 2/10
163/163 [==============================] - 125s 766ms/step - loss: 0.3986 - accuracy: 0.8301 - val_loss: 1.3121 - val_accuracy: 0.5227
Epoch 3/10
163/163 [==============================] - 124s 764ms/step - loss: 0.3612 - accuracy: 0.8482 - val_loss: 0.8191 - val_accuracy: 0.7227
Epoch 4/10
163/163 [==============================] - 122s 746ms/step - loss: 0.4265 - accuracy: 0.8395 - val_loss: 0.7040 - val_accuracy: 0.7136
Epoch 5/10
163/163 [==============================] - 124s 758ms/step - loss: 0.3261 - accuracy: 0.8729 - val_loss: 0.7197 - val_accuracy: 0.7045
Epoch 6/10
163/163 [==============================] - 122s 747ms/step - loss: 0.3242 - accuracy: 0.8669 - val_loss: 0.7621 - val_accuracy: 0.7318
Epoch 7/10
163/163 [==============================] - 124s 760ms/step - loss: 0.3071 - accuracy: 0.8767 - val_loss: 1.0038 - 

In [ ]:
!pip install gradio
import gradio as gr
import cv2
import numpy as np
img_size = 224
def load_and_preprocess_image(img):
    img = np.array(img)
    img = cv2.resize(img, (img_size, img_size))
    img = img.reshape(-1, img_size, img_size, 3)
    img = img / 255.0
    return img

def image_classifier(img):
    # Preprocess the image
    img = load_and_preprocess_image(img)
    # Perform prediction
    prediction = model.predict(img)[0]  # Assuming your model outputs a single probability
    print(prediction)
    # Return the prediction
    return {'PNEUMONIA': prediction[1], 'NORMAL': prediction[0]}  # Invert the prediction for Gradio

# Create Gradio interface
demo = gr.Interface(fn=image_classifier, inputs="image", outputs="label")
demo.launch()
# Launch the interface
test_loss, test_acc = model.evaluate(test_set, verbose=2)
print('Test accuracy:', test_acc)
print('Test loss:',test_loss)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ca8b826964e0a55ac9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 2s 2s/step
[0.35017192 0.6498281 ]
1/1 [==============================] - 0s 22ms/step
[0.32254004 0.6774599 ]
1/1 [==============================] - 0s 23ms/step
[9.9978966e-01 2.1032681e-04]
20/20 - 170s - loss: 0.5100 - accuracy: 0.7853 - 170s/epoch - 9s/step
Test accuracy: 0.7852563858032227
Test loss: 0.5100473165512085
